In [360]:
import pandas as pd
from datetime import date, timedelta
import untangle # xml
import requests # json
import re # regular expressions

In [361]:
# Configuration
today = date.today()
yesterday = date.today() - timedelta(days=1)

our_utla_codes = [
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E06000051', # Shropshire
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
    'E06000020', # Telford and Wrekin
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
    'E10000034', # Worcestershire
]
utlacodes_expressandstar = [
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
]
utlacodes_blackcountry = [
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
]
utlacodes_blackcountryandbirmingham = [
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
]
utlacodes_sandwellandbirmingham = [
    'E08000025', # Birmingham
    'E08000028', # Sandwell
]
utlacodes_staffordshire = [
    'E10000028', # Staffordshire
]
utlacodes_wolverhampton = [
    'E08000031', # Wolverhampton
]
utlacodes_walsall = [
    'E08000030', # Walsall
]
utlacodes_dudley = [
    'E08000027', # Dudley
]
utlacodes_sandwell = [
    'E08000028', # Sandwell
]
utlacodes_birmingham = [
    'E08000025', # Birmingham
]
utlacodes_stoke = [
    'E06000021', # Stoke-on-Trent
]
utlacodes_staffordshireandstoke = [
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
]
utlacodes_worcestershire = [
    'E10000034', # Worcestershire
]

In [362]:
# Pull the XML and traverse to the blobs
xmlblobs = untangle.parse(
    'https://publicdashacc.blob.core.windows.net/publicdata?restype=container&comp=list'
).EnumerationResults.Blobs.Blob

In [363]:
# Extract the cdata of the names, filter to data_000….json, convert to list, sort descending
jsons = sorted(
    list(
        filter(
            lambda b: re.search('^data_[0-9]+.json$',b),
            map(
                lambda b: b.Name.cdata,
                xmlblobs
            )
        )
    ),reverse=False
)

In [364]:
# Grab the last JSON in the sorted list, append it to the URL
latest_json_url = 'https://c19pub.azureedge.net/' + jsons[-1]

In [365]:
# Pull the JSON and convert to a Python object
json = requests.get(latest_json_url).json()

In [366]:
# Filter to MNA's UTLAs, populate utla_data
utla_data = []
for (code, data) in json['utlas'].items():
    if code in our_utla_codes:
        for day in data['dailyConfirmedCases']:
            utla_data.append([
                code,
                data['name']['value'],
                day['date'],
                day['value']
            ])

In [367]:
# Create a pandas dataframe from utla_data
our_utlas = pd.DataFrame(data=utla_data, columns=['code','name','date','new cases'])

In [368]:
def get_latest_cases(codes=False,df=our_utlas.copy()):
    if codes:
            df = df[df.code.isin(codes)]
            
    df['cumulative cases'] = df.groupby('code').cumsum()
    if codes:
            df = df.groupby('date', as_index=False).sum()
            
    max_date = df['date'].max()
    df = df[df['date'] == max_date]
    return df